In [ ]:
#| default_exp bbox_func/reg_loss

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export 
import torch
import torch.nn.functional as F
import fastcore.all as fc
from enum import Enum
from torch.nn.modules.loss import _Loss
from typing import Union
from voxdet.bbox_func import bbox_iou #import cal_diou_pair, cal_giou_pair

In [ ]:
import fastcore.all as fc

In [ ]:
#| export 
class StrEnum(str, Enum):
    """Enum subclass that converts its value to a string."""
    def __str__(self): return self.value
    def __repr__(self): return self.value

In [ ]:
#| export 
class LossReduction(StrEnum):
    NONE = "none"
    MEAN = "mean"
    SUM = "sum"

In [ ]:
fc.all_equal(list(LossReduction),  ["none", "mean", "sum"])
fc.eq(LossReduction.NONE, "none")
fc.eq(LossReduction.MEAN, "mean")

True

In [ ]:
#| export 
COMPUTE_DTYPE = torch.float32

## IOU Loss + L1 loss
TODO: write documentation
Compute the Distance intersection over union (DIoU) loss of a pair of boxes.
- The two inputs should have the same shape. diou_loss = 1.0 - diou
- The range of DIoU is (-1.0, 1.0]. Thus the range of DIoU loss is [0.0, 2.0).

In [ ]:
#| export 
class RegLoss(_Loss):
    def __init__(self, \
                 reduction: Union[LossReduction, str] = LossReduction.MEAN, \
                 iou_loss: str="diou", \
                 l1_loss: bool= True) -> None:
        super().__init__(reduction=LossReduction(reduction).value)
        fc.store_attr()
        
    
    def forward(self, inputs: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        if target.shape != inputs.shape:raise ValueError(f"ground truth has different shape ({target.shape}) from input ({input.shape})")
        box_dtype = inputs.dtype
        iou = getattr(bbox_iou, f'cal_{self.iou_loss}_pair')(target.to(dtype=COMPUTE_DTYPE), \
                                                             inputs.to(dtype=COMPUTE_DTYPE)) # type: ignore
        iou_loss= self._reduce(1.0 - iou).to(box_dtype)
        if self.l1_loss:
            l1_loss= F.l1_loss(target.to(dtype=COMPUTE_DTYPE), inputs.to(dtype=COMPUTE_DTYPE))
            l1_loss = self._reduce(l1_loss).to(box_dtype)
            return iou_loss, l1_loss 
        return iou_loss
    
    def _reduce(self, loss):
        if self.reduction == LossReduction.MEAN.value: loss = loss.mean()
        elif self.reduction == LossReduction.SUM.value: loss = loss.sum()
        elif self.reduction == LossReduction.NONE.value: pass
        else: raise ValueError(f'Unsupported reduction: {self.reduction}, available options are ["mean", "sum", "none"].')
        return loss

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()